In [ ]:
%%capture
!pip install pydub pyht
!pip install -U g4f[all]

In [ ]:
import random
from pydub import AudioSegment
from io import BytesIO
import requests
from tqdm.notebook import tqdm
import IPython
import json

from pyht import Client
from pyht.client import TTSOptions
import re

import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters


import nest_asyncio
import g4f
nest_asyncio.apply()

import shutil
from tqdm.notebook import tqdm
import pickle
import os

In [ ]:
abbreviations = ["etc", "err", "et", "ex", "e.g", "fin", "i.e", "vs", "N.B", "AD", "BC", "AM", "PM", "P.S", "mph", "kph", "ft", "lb", "oz", "pt", "in", "sec", "g", "cm", "qt"]

playht_url = "https://api.play.ht/api/v2/tts"

# GPT

In [ ]:
def find_acitve_gpt():
  active_providers = []
  providers = dir(g4f.Provider)
  for provider in providers:
    provider = getattr(g4f.Provider, provider)
    try:
      response = g4f.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": "Tell me your model name"}],
      provider=provider
      )
      print(response)

      if response != '':
        active_providers.append(provider)
    except:
      continue
  return active_providers

In [ ]:
def prompt(text):
  response = g4f.ChatCompletion.create(
    model=g4f.models.mixtral_8x7b,
    messages=[{"role": "user", "content": "I need you to translate from Russian to English and rewrite my text for 'speech text' for narrator and to make it more engaging, but you should sound natural and in casual style, also revise text for proper spelling and grammar.Send me 'Here's the translation and rewrite of the text for a speech:' and provide your answer"},
              {'role':'assistant','content':"Sure, I can help with that. Please provide the Russian text you'd like me to translate and rewrite for a more engaging, casual style."},
              {'role':'user','content':text}],
    #provider=g4f.Provider.Liaobots
)

  return response

# Voice

In [ ]:
def save_api():

  with open('apis.pkl', "wb") as file:
    # Use pickle to dump the list to the file
     pickle.dump(APIs, file)

In [ ]:
def combine_sentences(sentences):
    combined_sentences = []
    current_length = 0
    current_sentence = ''

    for sentence in sentences:
        if current_length + len(sentence) < 2000:
            current_sentence += sentence
            current_length += len(sentence)
        else:
            combined_sentences.append(current_sentence.strip())
            current_sentence = sentence
            current_length = len(sentence)

    # Добавляем оставшееся предложение
    if current_sentence:
        combined_sentences.append(current_sentence.strip())

    return combined_sentences

In [ ]:
def get_payload(text):
  payload = {
      "text": text,
      "voice": "s3://voice-cloning-zero-shot/4bc2279c-ec2e-471f-9369-faf5dac29321/original/manifest.json",
      "output_format": "mp3",
      "voice_engine": "PlayHT2.0",
      "temperature": 0.333,
      "seed": 42}
  return payload

In [ ]:
def get_headers(api,user_id):
  headers = {
    "accept": "text/event-stream",
    "content-type": "application/json",
    "AUTHORIZATION": api,
    "X-USER-ID": user_id}
  return headers

In [ ]:
def api_checker(text):

  for API in APIs:
    if len(text) + 1 < APIs[API][1]:
      api = API
      url = "https://api.play.ht/api/v2/tts"

      payload = {
          "text": "h",
          "voice": "s3://voice-cloning-zero-shot/d9ff78ba-d016-47f6-b0ef-dd630f59414e/female-cs/manifest.json",
          "output_format": "mp3",
          "voice_engine": "PlayHT2.0"
      }
      headers = {
          "accept": "text/event-stream",
          "content-type": "application/json",
          "AUTHORIZATION": api,
          "X-USER-ID": APIs[api][0]}

      response = requests.post(url, json=payload, headers=headers)

      if response.status_code == 200:
        APIs[api][1] -= 1
        print(API, APIs[api][1] - len(text) )
        return api
      else:
        print(f'NOT WORKING {API}, {response.status_code}')
        continue

In [ ]:
def TTS(text):
  #название файла
  path = text.split()[0]+'.wav'

  # Создаем токенизатор предложений с нашими параметрами
  punkt_param = PunktParameters()
  punkt_param.abbrev_types = set(abbreviations)
  tokenizer = PunktSentenceTokenizer(punkt_param)
  # Разбиваем текст на предложения
  sentences = tokenizer.tokenize(text)
  combined_sentences = combine_sentences(sentences)

  api = api_checker(text)
  audio_files = []
  for sentence in tqdm(sentences):


    #создание headers
    user_id = APIs[api][0]
    headers = get_headers(api,user_id)

    # создаем настройки для синтеза голоса, параметр только предложение
    payload = get_payload(sentence)
    make_audio = requests.post(playht_url, json=payload, headers=headers)

    audio_url = make_audio.text.split('"url":"')[1].split('","duration"')[0]
    APIs[api][1] -= len(sentences)

    download_audio = requests.get(audio_url)
    audio_bytes = BytesIO(download_audio.content)
    audio_file = AudioSegment.from_file(audio_bytes)
    audio_files.append(audio_file)

  return merge_audio(audio_files,path)

In [ ]:
def merge_audio(audio_files,path):

  # Создание сегмента тишины продолжительностью 1 секунда
  silence = AudioSegment.silent(duration=1)

  # Создание списка сегментов аудиофайлов
  audio_segments = [AudioSegment.from_mono_audiosegments(file) for file in audio_files]

  # Создание списка для склеенных сегментов
  merged_segments = []

  # Добавление первого сегмента
  merged_segments.append(audio_segments[0])

  # Добавление паузы и остальных аудиофайлов
  for segment in tqdm(audio_segments[1:]):
    merged_segments.extend([silence, segment])

  # Склеивание аудиофайлов
  merged_audio = sum(merged_segments)

  # Сохранение склеенного аудиофайла
  merged_audio.export(path, format="wav")
  return path

In [ ]:
!git clone https://github.com/PaleBill/stepik_abuse
!cd stepik_abuse

fatal: destination path 'stepik_abuse' already exists and is not an empty directory.


In [ ]:
abbreviations = ["etc", "err", "et", "ex", "e.g", "fin", "i.e", "vs", "N.B", "AD", "BC", "AM", "PM", "P.S", "mph", "kph", "ft", "lb", "oz", "pt", "in", "sec", "g", "cm", "qt"]

In [ ]:
keys = '''fahoweg765@mnsaf.com;Vn8hNRq*;0FzMG5HDWlfA7y8cuzrUr4Kjjlz2;1fb68c8352e7401ab9d23736be63e7fb
lafiw72077@mnsaf.com;u3Q=9z!%;cohjCRHXy7PFfPL9TBy0HlslDcf2;ff5b56b781154f4b85aaa347bc2ddbfd
cofopey509@nimadir.com;.5]^WUPU;gBFVYS1MuHVRz850luXkzph4l1D2;3dc9df0b56ff4380af875a5adc7c2cfc
wogaya1695@mnsaf.com;99~YmHNm;EOupbZitomf2FZezAi4rq9YBbv62;18c08454727f4733b4986c129c0ce416
kitici5534@mnsaf.com;~.wmyD0U;r6WO17KzCkhKHewOL7Hs5PSBKA52;979a04ffed504b96bd08e34145d8c383
jijare6794@nimadir.com;mA3W!m*.;nBlVzGRM1uRRiqG57wGjct8RVio1;3a3b7b93ce484ecd8674576a9db632b5;8800
natox66339@nimadir.com;%Gwq@4t_;4h41ZvnBPWcj5jNM8MXpXRhPCFt2;ace176168c21484291b1d81c9c5f260f
pasikoc180@ikumaru.com;8}J^mQjo;9HZ3FjizglQrozMsuwbs1uFaSuS2;e8cd439492bf476e858e9690dc3606c5
pogale2843@mnsaf.com;u39+W.@j;wUuBqR6JGPcMqkt57GuyFchhTWK2;ca0a82de967e48a48e6d268fe9b87554
yagoviv956@evimzo.com;U}vWKyQ9;Q6Q5i6aHWnfW2y6wIZwd5wvtEHV2;ed135450e7394bf28b61cac3981e7764
mibew89189@ekposta.com;FB~sGH4j;0n8bYe4reHZe8LLAllyZivbQfBC3;793692d016b7448988dc99ab5a005657
disin49696@evimzo.com;Deoj1)JN;jrGpywZs3aeaE0BDiZIuA3CyclN2;7da4bf3af09a462b8080d08bc836026f
jojinic471@felibg.com;a49q3sq!;8RJOODlZ6EfiypY9oq52nBkBh8e2;003ba0139356408488d572cbadfb1a98
vayeco7617@ekposta.com;v^)u39]U;FEnqNtLhY5ZzQKj4y2zneWUxyIs1;c05614aa55c941b0a234c8b0b4590c20'''.split('\n')


# Playground


## Translation Part

In [ ]:
!git clone https://github.com/PaleBill/stepik_abuse.git

Cloning into 'stepik_abuse'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 77
Receiving objects: 100% (78/78), 864.33 KiB | 9.50 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [ ]:
PATH = '/content/stepik_abuse/ID_COURSE_63054/'


In [ ]:
import json

In [ ]:
main_dir = os.listdir(PATH)
pathes = []
for dir in main_dir:
  lessons = os.listdir(PATH+dir)
  for lesson in lessons:
    new_path = PATH + dir + '/' + lesson
    for text in os.listdir(new_path):
      pathes.append(new_path+'/'+text)

In [ ]:
texts = {}
failed_json = []
for path in pathes:
  with open(path, 'r', encoding='UTF-8') as f:
      data = json.load(f)
  try:
    text = data[0]['steps'][0]['text']['text']
    texts[path] = text
  except:
    failed_json.append(path)
    continue

In [ ]:
preprocessed = {}
failed_text = {}
for path in tqdm(pathes):
  try:
    save_path = f'{path[:-12]}/translation_{path[-18:-12]}.txt'
    translation = prompt(texts[path])
    with open(save_path,'w') as f:
      f.write(translation)
    preprocessed[save_path] = translation
  except Exception as e:
    failed_text[path] = e

  0%|          | 0/22 [00:00<?, ?it/s]

## Voice part